In [1]:
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
train_es = conll2002.iob_sents('esp.train') # Train
dev_es = conll2002.iob_sents('esp.testa') # Dev
test_es =conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
dev_ned = conll2002.iob_sents('ned.testa') # Dev
test_ned =conll2002.iob_sents('ned.testb') # Test

data = {'spanish': (train_es, dev_es, test_es),
        'dutch': (train_ned, dev_ned, test_ned)}

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [2]:
train = conll2002.tagged_sents('esp.train')
test = conll2002.tagged_sents('esp.testb')
print(len(train))

8323


In [10]:
def gazetters():
    """
    Define a set of gazetteers for named entity recognition.
    These gazetteers include common locations, person names, and organizations.
    """

    # List of locations, person names, and organizations
    locations_list = {
        # Ciudades principales (España y Países Bajos)
        "madrid", "barcelona", "valencia", "sevilla", "zaragoza", "bilbao", "málaga", "murcia", "palma",
        "amsterdam", "rotterdam", "utrecht", "eindhoven", "groningen", "nijmegen",

        # Regiones y provincias
        "andalucía", "cataluña", "galicia", "castilla", "canarias", "baleares",
        "noord-holland", "zuid-holland", "gelderland", "limburg",

        # Países (hispanohablantes + relevantes cercanos)
        "españa", "méxico", "argentina", "colombia", "perú", "chile", "ecuador",
        "nederland", "belgië", "duitsland", "francia", "italia", "portugal", "reino unido", "estados unidos",

        # Geografía física
        "mediterráneo", "atlántico", "pirineos", "cantábrico", "guadalquivir", "ebro",
        "noordzee", "rijn", "maas", "veluwe"
    }

    person_names_list = {
        # Nombres masculinos comunes
        "juan", "josé", "david", "javier", "miguel", "pedro", "sergio", "pablo", "antonio",
        "jan", "peter", "willem", "tim", "thomas", "jeroen", "lucas", "kees",

        # Nombres femeninos comunes
        "maría", "ana", "isabel", "laura", "marta", "lucía", "paula", "cristina",
        "maria", "johanna", "emma", "sophie", "lisa", "lotte", "iris", "eva",

        # Apellidos comunes
        "garcía", "gonzález", "rodríguez", "fernández", "lópez", "sánchez", "martínez",
        "de jong", "jansen", "de vries", "van den berg", "bakker", "visser", "meijer"
    }

    organizations_list = {
        # Empresas destacadas
        "telefónica", "bbva", "iberdrola", "mercadona", "inditex", "seat",
        "philips", "shell", "unilever", "heineken", "ing", "asml",

        # Instituciones gubernamentales
        "gobierno", "ministerio", "ayuntamiento", "generalitat", "policía nacional", "guardia civil",
        "regering", "ministerie", "gemeente", "belastingdienst", "politie", "koninklijke marechaussee",

        # Educación y cultura
        "universidad", "museo del prado", "csic", "colegio", "hospital", "asociación", "fundación",
        "universiteit", "hogeschool", "tno", "knaw", "rijksmuseum", "omroep", "kvk"
    }

    return locations_list, person_names_list, organizations_list

In [11]:
import re
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemma(word):
    return lemmatizer.lemmatize(word)

def pos_tag(word):
    tagged = nltk.pos_tag([word])
    return tagged[0][1]

def generate_features(feature_selection):
    def feature_function(sentence, index):
        features = {"bias": 1.0}  # Bias term for the feature vector
        word = sentence[index][0]  # Always get the first element (word)
        
        # Características básicas de la palabra
        if feature_selection.get("word_form", True):
            features.update({
                "word": word,
                "word.lower": word.lower(),
            })

        # Características morfológicas
        if feature_selection.get("morphology", True):
            features.update({
                "word.istitle": word.istitle(),
                "word.isupper": word.isupper(),
                "word.islower": word.islower(),
                "word.isdigit": word.isdigit(),
                "has_digit": any(c.isdigit() for c in word),
                "capitals_inside": any(c.isupper() for c in word[1:]),
                "has_symbol": not word.isalnum()
            })
        
        # POS tagging y lemas
        if feature_selection.get("lemma_pos_tags", True):
            features["pos_tag"] = pos_tag(word)
            features["lemma"] = lemma(word)
        
        # Prefijos y sufijos
        if feature_selection.get("prefix_suffix", True):
            features.update({
                "prefix3": word[:3],
                "suffix3": word[-3:],
                "prefix2": word[:2],
                "suffix2": word[-2:],
            })
        
        # Longitud de la palabra
        if feature_selection.get("length", True):
            features["length"] = len(word)
        
        # Posición en la oración
        if feature_selection.get("position", True):
            features.update({
                "position": index,
                "is_first": index == 0,
                "is_last": index == len(sentence)-1
            })
        
        # Contexto circundante
        if feature_selection.get("context", True):
            if index > 0:
                prev_word = sentence[index-1][0]
                features.update({
                    "prev_word.lower": prev_word.lower(),
                    "prev_word.istitle": prev_word.istitle(),
                    "prev_word.isupper": prev_word.isupper(),
                    "prev_word.isdigit": prev_word.isdigit(),
                    "prev_word.istitle": prev_word.istitle()})
                
                if feature_selection.get("lemma_pos_tags", True):
                    features["prev_lemma"] = lemma(prev_word)
                    features["prev_pos_tag"] = pos_tag(prev_word)

            if index < len(sentence)-1:
                next_word = sentence[index+1][0]
                features.update({
                    "next_word.lower": next_word.lower(),
                    "next_word.istitle": next_word.istitle(),
                    "next_word.isupper": next_word.isupper(),
                    "next_word.isdigit": next_word.isdigit(),
                    "next_word.istitle": next_word.istitle()})
                
                if feature_selection.get("lemma_pos_tags", True):
                    features["next_lemma"] = lemma(next_word)
                    features["next_pos_tag"] = pos_tag(next_word)
        
        # Patrones de fecha y hora
        if feature_selection.get("date_time_patterns", True):
            date_pattern = r"\d{1,2}[/-]\d{1,2}([/-]\d{2,4})?"
            time_pattern = r"\d{1,2}:\d{2}(:\d{2})?"
            features["has_date_pattern"] = bool(re.search(date_pattern, word))
            features["has_time_pattern"] = bool(re.search(time_pattern, word))

        # Caracteres especiales y símbolos
        if feature_selection.get("symbol_patterns", True):
            features.update({
            "has_hyphen": "-" in word,
            "has_dot": "." in word,
            "has_comma": "," in word,
            "has_slash": "/" in word,
            "has_percent": "%" in word,
            "has_currency": any(c in word for c in "$€£¥"),
            "has_at": "@" in word
            })

                # En generate_features, añadir:
        if feature_selection.get("gazetteers", True):
            locations_list, person_names_list, organizations_list = gazetters()
            features.update({
                "in_locations": word.lower() in locations_list,
                "in_person_names": word.lower() in person_names_list,
                "in_organizations": word.lower() in organizations_list
            })
        
        return features
    
    return feature_function

In [12]:
feature_selection = {
    "word_form": True,           # Basic word characteristics
    "prefix_suffix": True,       # Prefixes and suffixes
    "morphology": True,          # Morphological features
    "context": True,             # Surrounding words
    "lemma_pos_tags": True,      # POS tagging and lemmas
    "date_time_patterns": True,  # Date and time patterns
    "symbol_patterns": True,     # Special characters
    "length": True,              # Word length features
    "position": True,            # Position features
    "gazetteers": True           # Gazetteer features
}

ct = nltk.tag.CRFTagger(feature_func=generate_features(feature_selection))
ct.train(train[:100], "nooooooolapolitziiaa.mdl")
ct.accuracy(test)

0.6670482991481187

In [ ]:
import nltk
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from typing import List, Tuple, Optional, Callable
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('conll2002')

class CRFModel:
    # Clase proporcionada por el usuario (como en el enunciado)
    def __init__(self, train, test, model_file: Optional[str] = None, features: Optional[Callable] = None, encoding: str = 'bio') -> None:
        self.model = CRFTagger(feature_func=features)
        self.model_file = model_file
        self.encoding = encoding
        self.train_data = self.transformar_dades(train)
        self.test_data = self.transformar_dades(test)

    def transformar_dades(self, data):
        if self.encoding == 'io':
            data_encoded = [self.to_io(sent) for sent in data]
        elif self.encoding == 'bioes':
            data_encoded = [self.to_bioes(sent) for sent in data]
        else:
            data_encoded = data
        return data_encoded

    def train(self, train_data: List[List[Tuple[str, str]]]) -> None:
        self.model.train(train_data, self.model_file)

    def predict(self, sents: List[List[Tuple[str, str]]]) -> List[List[Tuple[str, str]]]:
        if self.model_file:
            self.model.set_model_file(self.model_file)
        words_test = [[word for (word, _) in sent] for sent in sents]
        return self.model.tag_sents(words_test)

    # Resto de métodos (to_bioes, to_io, extraccio_entitats, evaluacio_entitats, matriu_confusio) como en el enunciado

class NERCRF:
    def __init__(self, language: str = 'esp', encoding: str = 'bio', feature_func: Optional[Callable] = None, model_file: Optional[str] = None):
        self.language = language
        self.encoding = encoding
        self.feature_func = feature_func
        self.model_file = model_file

        # Cargar datos
        self.train_raw = self._load_conll_data(language+'.train')
        self.dev_raw = self._load_conll_data(language+'.testa')
        self.test_raw = self._load_conll_data(language+'.testb')

        # Inicializar modelo CRF
        self.crf_model = CRFModel(
            train=self.train_raw,
            test=self.test_raw,
            model_file=self.model_file,
            features=self.feature_func,
            encoding=self.encoding
        )

    def _load_conll_data(self, split: str) -> List[List[Tuple[str, str]]]:
        sents = conll2002.iob_sents(f'{self.language}.{split}')
        return [[(word, tag) for (word, _, tag) in sent] for sent in sents]

    def train_model(self):
        self.crf_model.train(self.crf_model.train_data)

    def evaluate(self, split: str = 'dev') -> Tuple[float, float, float]:
        if split == 'dev':
            data = self.dev_raw
        elif split == 'test':
            data = self.test_raw
        else:
            raise ValueError("split debe ser 'dev' o 'test'")

        # Transformar datos al encoding correcto
        data_encoded = self.crf_model.transformar_dades(data)

        # Predecir etiquetas
        predicted = self.crf_model.predict(data_encoded)

        # Extraer entidades
        y_true = []
        for sent in data_encoded:
            entities = self.crf_model.extraccio_entitats([sent])
            y_true.extend(entities)

        y_pred = []
        for sent in predicted:
            entities = self.crf_model.extraccio_entitats([sent])
            y_pred.extend(entities)

        # Evaluar
        recall, precision, f1 = self.crf_model.evaluacio_entitats(y_true, y_pred)
        return recall, precision, f1

    def predict_sentence(self, sentence: List[str]) -> List[Tuple[str, str]]:
        formatted_sent = [[(word, 'O') for word in sentence]]
        predicted = self.crf_model.predict(formatted_sent)
        return predicted[0]

# Ejemplo de uso
if __name__ == "__main__":
    # Función de características de ejemplo
    def custom_features(sentence, index):
        word = sentence[index][0]
        features = {
            'word': word,
            'word.lower()': word.lower(),
            'prefix3': word[:3],
            'suffix3': word[-3:],
            'prev_word': sentence[index-1][0] if index > 0 else '<START>',
            'next_word': sentence[index+1][0] if index < len(sentence)-1 else '<END>',
            'is_capitalized': word[0].isupper(),
        }
        return features

    # Entrenar modelo para español con BIOES
    spanish_model = NERCRF(
        language='esp',
        encoding='bioes',
        feature_func=custom_features,
        model_file='esp_model.crf'
    )
    spanish_model.train_model()
    recall, precision, f1 = spanish_model.evaluate('dev')
    print(f"Resultados para español (BIOES):\nRecall: {recall:.2f}, Precision: {precision:.2f}, F1: {f1:.2f}")

    # Entrenar modelo para neerlandés con IO
    dutch_model = NERCRF(
        language='ned',
        encoding='io',
        feature_func=custom_features,
        model_file='ned_model.crf'
    )
    dutch_model.train_model()
    recall, precision, f1 = dutch_model.evaluate('test')
    print(f"Resultados para neerlandés (IO):\nRecall: {recall:.2f}, Precision: {precision:.2f}, F1: {f1:.2f}")